## Downloading The Jigsaw Unintended Bias in Toxicity Classification Dataset

In [1]:
!pip install kaggle
import pandas as pd
from google.colab import files

In [2]:
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"maysfar","key":"95245171d4da79c1b6ea29150f1d4544"}'}

In [3]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [4]:
!kaggle competitions download -c jigsaw-unintended-bias-in-toxicity-classification -p /content/jigsaw_data
!unzip /content/jigsaw_data/jigsaw-unintended-bias-in-toxicity-classification.zip -d /content/jigsaw_data

 90% 649M/723M [00:03<00:01, 46.5MB/s]
100% 723M/723M [00:03<00:00, 193MB/s] 
Archive:  /content/jigsaw_data/jigsaw-unintended-bias-in-toxicity-classification.zip
  inflating: /content/jigsaw_data/all_data.csv  
  inflating: /content/jigsaw_data/identity_individual_annotations.csv  
  inflating: /content/jigsaw_data/sample_submission.csv  
  inflating: /content/jigsaw_data/test.csv  
  inflating: /content/jigsaw_data/test_private_expanded.csv  
  inflating: /content/jigsaw_data/test_public_expanded.csv  
  inflating: /content/jigsaw_data/toxicity_individual_annotations.csv  
  inflating: /content/jigsaw_data/train.csv  


In [5]:
import pandas as pd

In [6]:
df = pd.read_csv("/content/jigsaw_data/train.csv")

In [7]:
df.sample(5)

,id,target,comment_text,severe_toxicity,obscene,identity_attack,insult,threat,asian,atheist,...,article_id,rating,funny,wow,sad,likes,disagree,sexual_explicit,identity_annotator_count,toxicity_annotator_count
1281852,5680926,0.000000,Dave has Avery Shenfeld saying: “While inflati...,0.000000,0.000000,0.0,0.000000,0.000000,NaN,NaN,...,360329,approved,0,0,0,0,0,0.0,0,4
1679309,6180557,0.166667,Another suicide. When is the City going to put...,0.000000,0.000000,0.0,0.000000,0.166667,0.0,0.0,...,391025,approved,0,1,0,2,6,0.0,5,6
1783350,6307346,0.166667,And that is just nonsense.,0.000000,0.000000,0.0,0.166667,0.000000,NaN,NaN,...,397841,approved,0,0,0,9,4,0.0,0,6
243474,541572,0.000000,It would be better if you traced your own comm...,0.000000,0.000000,0.0,0.000000,0.000000,NaN,NaN,...,148829,approved,0,0,0,0,0,0.0,0,4
1720100,6230801,0.808824,Her life is turned upside down and your best i...,0.014706,0.264706,0.0,0.735294,0.014706,NaN,NaN,...,393578,approved,0,0,0,0,0,0.0,0,68


## Cleaning

In [8]:
data = df[["comment_text", "target", "male", "female"]].copy()

In [9]:
len(data)

1804874

In [10]:
data.sample(5)

,comment_text,target,male,female
334069,Unbelievable! Toddler finds non-locked-up gun....,0.000000,NaN,NaN
608265,It's disappointing that the pro-aborts were gi...,0.166667,0.0,1.0
221410,"the bs the libs live by, the mother once spoke...",0.700000,0.0,0.0
856777,"They do have security guards in the store, but...",0.000000,0.0,0.0
1735625,Sounds like some judges need to be voted out.,0.000000,NaN,NaN


Replacing NaN values with 0.0

In [11]:
data[["male", "female"]] = data[["male", "female"]].fillna(0.0)

Dropping duplicates

In [12]:
data = data.drop_duplicates().reset_index(drop=True)

In [13]:
len(data)

1786304

Strip leading/trailing whitespace and normalize Unicode (NFC)

In [14]:
import unicodedata
def strip_and_normalize_nfc(text: str) -> str:
    if not isinstance(text, str):
        text = "" if text is None else str(text)
    text = text.strip()
    return unicodedata.normalize("NFC", text)

Replace URLs with <URL> and user handles with <USER>

In [15]:
import re
URL_RE  = re.compile(r'((?:https?://|http?://|www\.)\S+)', flags=re.IGNORECASE)
USER_RE = re.compile(r'(?<!\w)@\w+')

def replace_urls_and_users(text: str) -> str:
    text = URL_RE.sub("<URL>", text)
    text = USER_RE.sub("<USER>", text)
    return text

Remove \n and similar, then collapse excess whitespace to single spaces

In [16]:
WS_RE = re.compile(r"\s+")

def remove_newlines_and_collapse_ws(text: str) -> str:
    # replace any whitespace run (including \n, \t) with a single space
    return WS_RE.sub(" ", text).strip()

Sanity checks:

In [17]:
URL_RE  = re.compile(r'((?:https?://|http?://|www\.)\S+)', flags=re.IGNORECASE)
USER_RE = re.compile(r'(?<!\w)@\w+')
NL_RE = re.compile(r"\n")

url_idx = data["comment_text"].apply(lambda x: bool(URL_RE.search(str(x))))
user_idx = data["comment_text"].apply(lambda x: bool(USER_RE.search(str(x))))
nl_idx   = data["comment_text"].apply(lambda x: bool(NL_RE.search(str(x))))

In [18]:
example_url  = data.loc[url_idx].sample(1)
example_user = data.loc[user_idx].sample(1)
example_nl   = data.loc[nl_idx].sample(1)

In [19]:
print("Example with URL:")
print(example_url["comment_text"].values[0])
print("\n👉 After replacement:")
print(replace_urls_and_users(example_url["comment_text"].values[0]))

print("\n" + "="*80 + "\n")

print("Example with USER handle:")
print(example_user["comment_text"].values[0])
print("\n After replacement:")
print(replace_urls_and_users(example_user["comment_text"].values[0]))

print("\n" + "="*80 + "\n")

print("Example with NEWLINES:")
print(repr(example_nl["comment_text"].values[0]))
print("\n After cleaning:")
print(remove_newlines_and_collapse_ws(example_nl["comment_text"].values[0]))

Example with URL:
Renewables have become cheaper than fossil fuels in the last several years??????

Just not true, when you dig into the details. 

https://www.forbes.com/sites/uhenergy/2017/04/04/despite-claims-of-grid-parity-wind-and-solar-are-still-more-expensive-than-fossil-fuels/2/#464d13f13e00

If your statements were really true, there'd be no reason for government subsidies of the renewable industries. The invisible hand of the free market would do everything needed to replace fossil with renewable. Someday, this will likely happen, but we're not even close to that yet, especially with the expansion of oil supplies over the last few years.

"general consensus" is like saying "anonymous sources". No proof or evidence required.

👉 After replacement:
Renewables have become cheaper than fossil fuels in the last several years??????

Just not true, when you dig into the details. 

<URL>

If your statements were really true, there'd be no reason for government subsidies of the renewab

Cleaning:

In [20]:
# 1. strip + normalize
data["comment_text"] = data["comment_text"].apply(strip_and_normalize_nfc)

# 2. replace URLs and user handles
data["comment_text"] = data["comment_text"].apply(replace_urls_and_users)

# 3. remove newlines + collapse whitespace
data["comment_text"] = data["comment_text"].apply(remove_newlines_and_collapse_ws)

# 4. drop true row duplicates across all columns
data = data.drop_duplicates().reset_index(drop=True)

data = data.rename(columns={"comment_text": "comment"})

In [21]:
data = data.dropna(subset=["comment"])                       # drop NaN
data = data[data["comment"].str.strip().astype(bool)]        # drop empty/whitespace

In [22]:
data.sample(5)

,comment,target,male,female
349457,"Ha ha, funny. My voice was heard on Nov 8th. I...",0.000000,0.0,0.0
167384,Thanks JOEL. We'll consider that.,0.000000,0.0,0.0
851478,"""In today’s education system, in which budgets...",0.000000,0.0,0.0
1604405,"If you can get that through Congress, by all m...",0.741935,0.0,0.0
1295291,"Before saying yes Littleton voters, refer back...",0.166667,0.0,0.0


## Assigning identities

In [23]:
len(data)

1782961

In [24]:
both_data = data[(data["male"] > 0.5) & (data["female"] > 0.5)]
both_data.sample(5)

,comment,target,male,female
205017,"Exactly, the BILLION Dollar NFL. I'm sure all ...",0.0,1.0,1.00
1308127,"I have a STEM career. I am an Engineer, female...",0.0,1.0,1.00
962925,"Step away from fiction for a sec, and consider...",0.5,1.0,0.75
1662326,"I assume you would chose being a guy, that's t...",0.0,1.0,1.00
912297,The discrepancy would have sorted itself out w...,0.3,1.0,1.00


In [25]:
data.loc[719492]["comment"]

"I think it's precisely because of the developing world that Francis is moving at a snail's pace on the question of women's inclusion. The developing world is dominated by males and the Church competes directly with the Islamic world whose attitude towards women has just started moving fractionally on the concept of women's equality. Genital mutilation and honor killings are still way too prevalent. I wonder sometimes if the pace of change in the RCC isn't being dictated by the misogyny still so prevalent in the developing world and the best the Church is willing to do is show a somewhat 'kinder gentler face', of the same misogyny. Or as the GOP might call it, compassionate misogyny."

## For Binary Classification

In [26]:
data['target'] = (data['target'] >= 0.5).astype(int)

In [27]:
data.sample(5)

,comment,target,male,female
423431,You are the least imaginative person here. I w...,0,0.0,0.0
231818,"""If you can get people to understand the risk ...",0,0.0,0.0
810173,Enough of this corrupt bunch. Let's do a compl...,0,0.0,0.0
687001,We don't care.,0,0.0,0.0
1413986,"When are these culprits going to jail, an ordi...",0,0.0,0.0


# Data Augmentation

In [28]:
def comment_gender_swap(comment, gendered_word_pairs):
    # Sort keys by length (longest first) to avoid partial matches
    keys = sorted(gendered_word_pairs.keys(), key=len, reverse=True)

    # Build one regex that matches any of the gendered words
    pattern = re.compile(
        r'\b(?:' + '|'.join(map(re.escape, keys)) + r')\b',
        flags=re.IGNORECASE
    )

    def replace(m):
        word = m.group(0)
        base = gendered_word_pairs[word.lower()]

        if word.isupper():        # e.g., "HE" → "SHE"
            return base.upper()
        elif word[0].isupper():  # e.g., "He" → "She"
            return base.capitalize()
        else:                    # e.g., "he" → "she"
            return base.lower()

    return pattern.sub(replace, comment)

In [54]:
def comment_gender_mask(text, genderTerms, token="[GENDER]"):
    """
    String-level masking (like comment_gender_swap but replaces with a fixed token).
    """
    rx = _compile_gender_mask_regex_from_terms(genderTerms)
    return rx.sub(token, str(text))

In [29]:
def augment_with_gender_swap(df,pairs):

    # 1) produce swapped texts for all rows
    swapped_texts = df["comment"].astype(str).apply(lambda t: comment_gender_swap(t, pairs))

    # 2) find rows where text actually changed
    changed = ~swapped_texts.eq(df["comment"].astype(str))
    if not changed.any():
        return df.copy()

    # 3) take only changed rows, set new text
    aug = df.loc[changed].copy()
    aug["comment"] = swapped_texts.loc[changed].values

    # 4) swap numeric male/female values on those rows
    tmp = aug["male"].copy()
    aug["male"]   = aug["female"].values
    aug["female"] = tmp.values

    # 5) return original + augmented rows
    return pd.concat([df, aug], ignore_index=True)

In [56]:
def _compile_gender_mask_regex_from_terms(terms):
    """
    Build a single case-insensitive regex that matches any term
    (plus simple plural/possessive tails like s/es/'s).
    """
    if not terms:
        return re.compile(r"(?!x)x", flags=re.IGNORECASE)  # never matches
    # longest-first to avoid partial overlaps (e.g., 'herself' before 'her')
    vocab = sorted({t.lower() for t in terms}, key=len, reverse=True)
    pattern = r"\b(?:%s)(?:['’]s|s|es)?\b" % "|".join(map(re.escape, vocab))
    return re.compile(pattern, flags=re.IGNORECASE)

def augment_with_gender_mask(df, genderTerms, text_col="comment", token="[GENDER]"):
    """
    Returns original df + masked duplicates for rows whose text changed.
    - Uses the provided `terms` list (derived from your gendered pairs).
    - Zeros subgroup cols (male/female) on augmented rows.
    - No new columns are added.
    """
    #assert text_col in df.columns, f"Missing column: {text_col}"
    rx = _compile_gender_mask_regex_from_terms(genderTerms)

    # 1) produce masked texts for all rows
    masked_texts = df[text_col].astype(str).apply(lambda t: rx.sub(token, t))

    # 2) find rows where text actually changed
    changed = ~masked_texts.eq(df[text_col].astype(str))
    if not changed.any():
        return df.copy()

    # 3) take only changed rows, set new text
    aug = df.loc[changed].copy()
    aug[text_col] = masked_texts.loc[changed].values

    # 4) zero numeric male/female values on those rows (if present)
    for col in ("male", "female"):
        if col in aug.columns:
            aug[col] = 0

    # 5) return original + augmented rows
    return pd.concat([df, aug], ignore_index=True) #Not sure IF we want to have only masked or both


Example:

In [42]:
gendered_word_pairs = {
    # Pronouns
    "he": "she",
    "him": "her",
    "his": "hers",
    "himself": "herself",

    # Common people words
    "man": "woman",
    "men": "women",
    "boy": "girl",
    "boys": "girls",
    "guy": "girl",
    "guys": "girls",
    "dude": "chick",   # casual/slang
    "bro": "sis",
    "gentleman": "lady",

    # Family terms
    "dad": "mom",
    "father": "mother",
    "son": "daughter",
    "brother": "sister",
    "uncle": "aunt",
    "husband": "wife",
    "boyfriend": "girlfriend",
    "bf": "gf",

    # Roles / references (popular in slang)
    "king": "queen",
    "prince": "princess"
}

# add reverse pairs IN PLACE
for k, v in list(gendered_word_pairs.items()):
    gendered_word_pairs[v] = k

# Build terms from your existing dict (both keys and values)
genderTerms = sorted({str(k).lower() for k in gendered_word_pairs.keys()} |
               {str(v).lower() for v in gendered_word_pairs.values()})



In [59]:
subset = data[(data["target"] == 1)].copy().sample(3).reset_index(drop=True)
subset

,comment,target,male,female
0,If a person lacks the intelligence to go get a...,1,0.0,0.0
1,"The victim of child abuse by his father, then ...",1,0.0,0.0
2,Drumpf is an embarrassment. I heard Merkel's s...,1,0.0,0.0


In [60]:
swap_aug_subset = augment_with_gender_swap(subset, gendered_word_pairs)
mask_aug_subset = augment_with_gender_mask(subset, genderTerms)
print(swap_aug_subset)
print(mask_aug_subset)

                                             comment  target  male  female
0  If a person lacks the intelligence to go get a...       1   0.0     0.0
1  The victim of child abuse by his father, then ...       1   0.0     0.0
2  Drumpf is an embarrassment. I heard Merkel's s...       1   0.0     0.0
3  The victim of child abuse by hers mother, then...       1   0.0     0.0
4  Drumpf is an embarrassment. I heard Merkel's s...       1   0.0     0.0
                                             comment  target  male  female
0  If a person lacks the intelligence to go get a...       1   0.0     0.0
1  The victim of child abuse by his father, then ...       1   0.0     0.0
2  Drumpf is an embarrassment. I heard Merkel's s...       1   0.0     0.0
3  The victim of child abuse by [GENDER] [GENDER]...       1   0.0     0.0
4  Drumpf is an embarrassment. I heard Merkel's s...       1   0.0     0.0


In [33]:
subset.loc[1]["comment"]

'really? if you had any morality you would not be posting such nonsense, grow up'

In [64]:
print(swap_aug_subset.loc[1]["comment"])
print(mask_aug_subset.loc[3]["comment"])


The victim of child abuse by his father, then the last federal government, now we're told he has to be perfect or Canadians will be unhappy. Sick.
The victim of child abuse by [GENDER] [GENDER], then the last federal government, now we're told [GENDER] has to be perfect or Canadians will be unhappy. Sick.


In [57]:
print(comment_gender_swap("The woman's story and the women's rights are important.", gendered_word_pairs))
print(comment_gender_mask("The woman's story and the women's rights are important.", genderTerms))


The man's story and the men's rights are important.
The [GENDER] story and the [GENDER] rights are important.
